In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv("data.csv")

k6 = df[df["K"] == 6]

In [8]:
data_loss_points = k6.sample(50)

In [9]:
data_loss_x = []
data_loss_t = []
for i in range(50):
        data_loss_x.append(data_loss_points.iloc[i][2])
        data_loss_t.append(data_loss_points.iloc[i][1])

/tmp/ipykernel_27510/215239933.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_loss_x.append(data_loss_points.iloc[i][2])
/tmp/ipykernel_27510/215239933.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_loss_t.append(data_loss_points.iloc[i][1])


In [10]:
data_loss_x_tensor = torch.tensor(data_loss_x, dtype=torch.float32, device="cuda").view(-1,1)
data_loss_t_tensor = torch.tensor(data_loss_t, dtype=torch.float32, device="cuda").view(-1,1)

In [11]:
t_points = torch.linspace(0,10,1000,dtype=torch.float32,device="cuda",requires_grad=True).view(-1,1)

In [12]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1,40)
        self.fc2 = nn.Linear(40,40)
        self.fc3 = nn.Linear(40,40)
        self.fc4 = nn.Linear(40,40)   # Architecture given by Raissi et al.
        self.fc5 = nn.Linear(40,40)
        self.fc6 = nn.Linear(40,1)
        self.k = nn.Parameter(torch.tensor(10.0, dtype=torch.float32,requires_grad=True))
    def forward(self,x):
        x = self.fc1(x)
        x = F.tanh(x)
        x = self.fc2(x)
        x = F.tanh(x)
        x = self.fc3(x)
        x = F.tanh(x)
        x = self.fc4(x)
        x = F.tanh(x)
        x = self.fc5(x)
        x = F.tanh(x)
        x = self.fc6(x)
        return x

In [13]:
pinner = Model().to("cuda")

In [14]:
optimizer = torch.optim.Adam(pinner.parameters(),lr= 0.005)

In [15]:
datalosslist = []
iclosslist = []
phylosslist= []
total_losslist = []
k_list = []

In [16]:
def train(model,ts,optimizer,iteration):
    for i in tqdm(range(iteration)):
        model.zero_grad()
        outs = model(ts)

        #data loss
        data_out = model(data_loss_t_tensor)
        data_loss = ((data_out - data_loss_x_tensor)**2).mean()

        #i.c loss
        t_ic = torch.tensor([[0.0]], dtype=torch.float32, device="cuda", requires_grad=True)
        x_ic_pred = model(t_ic)
        dxdt0 = torch.autograd.grad(x_ic_pred,t_ic,retain_graph=True,grad_outputs=torch.ones_like(t_ic),create_graph=True)[0]
        ic_loss = ((x_ic_pred - 1)**2 + (dxdt0)**2).mean()

        #physics loss
        dxdt1 = torch.autograd.grad(outs,ts,retain_graph=True,grad_outputs=torch.ones_like(ts),create_graph=True)[0]
        d2xdt2 = torch.autograd.grad(dxdt1,ts,retain_graph=True,grad_outputs=torch.ones_like(ts),create_graph=True)[0]
        k = model.k
        phy_loss = (((d2xdt2) + (k * outs))**2).mean()

        #if i < 5000:
        #total loss
            # total_loss = data_loss + ic_loss
        # elif i == 5000:
          #  t_pointscpu = t_points.detach().cpu().numpy()
          #  x_model_cpu = pinner(t_points).detach().cpu().numpy()
          #  plt.plot(t_pointscpu,x_model_cpu)
          #  total_loss = data_loss + ic_loss
        # else:
        total_loss = data_loss + ic_loss + phy_loss
        # k_reg = 0.001 * (model.k**2)
        # total_loss += k_reg
        
        total_loss.backward()
        optimizer.step()

        if i % 100 == 0:
            datalosslist.append(data_loss.item())
            iclosslist.append(ic_loss.item())
            phylosslist.append(phy_loss.item())
            total_losslist.append(total_loss.item())
            k_list.append(k.item())

In [17]:
train(pinner,t_points,optimizer,20000)

  0%|          | 0/20000 [00:00<?, ?it/s]

/home/umut/projects/python/dl/dlvenv/lib/python3.12/site-packages/torch/autograd/graph.py:841: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:270.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [ ]:
plt.plot(k_list)

In [ ]:
plt.plot(datalosslist)

In [ ]:
plt.plot(phylosslist)

In [ ]:
plt.plot(total_losslist)

In [ ]:
t_pointscpu = t_points.detach().cpu().numpy()
x_model_cpu = pinner(t_points).detach().cpu().numpy()

In [ ]:
plt.plot(t_pointscpu,x_model_cpu)

In [ ]:
k_list

In [ ]:
datalosslist